In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading Data
df_train = pd.read_csv('../data/raw/train.csv')
df_store = pd.read_csv('../data/raw/store.csv')

/var/folders/wy/rd6_hk5s6xx5sy02hxjsz2gr0000gn/T/ipykernel_17520/4176508223.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('../data/raw/train.csv')


In [3]:
df_train['Date'] = pd.to_datetime(df_train['Date'])

df_train['Day'] = df_train['Date'].dt.day
df_train['Month'] = df_train['Date'].dt.month
df_train['Quarter'] = df_train['Date'].dt.quarter

In [4]:
df_train = df_train.drop('Id', axis=1)
df_train = pd.merge(df_train, df_store, on='Store')

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001599 entries, 0 to 1001598
Data columns (total 21 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1001599 non-null  int64         
 1   DayOfWeek                  1001599 non-null  int64         
 2   Date                       1001599 non-null  datetime64[ns]
 3   Sales                      1001599 non-null  int64         
 4   Customers                  1001599 non-null  int64         
 5   Open                       1001599 non-null  int64         
 6   Promo                      1001599 non-null  int64         
 7   StateHoliday               1001599 non-null  object        
 8   SchoolHoliday              1001599 non-null  int64         
 9   Day                        1001599 non-null  int32         
 10  Month                      1001599 non-null  int32         
 11  Quarter                    1001599 no

In [7]:
df_train['StateHoliday_0'] = np.where((df_train['StateHoliday'] == 0) | (df_train['StateHoliday'] == '0'), 1, 0)
df_train['StateHoliday_a'] = np.where(df_train['StateHoliday'] == 'a', 1, 0)
df_train['StateHoliday_b'] = np.where(df_train['StateHoliday'] == 'b', 1, 0)
df_train['StateHoliday_c'] = np.where(df_train['StateHoliday'] == 'c', 1, 0)

df_train = df_train.drop(['StateHoliday'], axis=1)

In [8]:
df_train['StoreType_a'] = np.where(df_train['StoreType'] == 'a', 1, 0)
df_train['StoreType_b'] = np.where(df_train['StoreType'] == 'b', 1, 0)
df_train['StoreType_c'] = np.where(df_train['StoreType'] == 'c', 1, 0)
df_train['StoreType_d'] = np.where(df_train['StoreType'] == 'd', 1, 0)
df_train = df_train.drop(['StoreType'], axis=1)

df_train['Assortment'] = np.where(df_train['Assortment'] == 'a', 1, 0)
df_train['Assortment'] = np.where(df_train['Assortment'] == 'b', 1, 0)
df_train['Assortment'] = np.where(df_train['Assortment'] == 'c', 1, 0)
df_train = df_train.drop(['Assortment'], axis=1)

In [9]:
df_train['PromoInterval_Jan'] = np.where(df_train['PromoInterval'].str.contains('Jan'), 1, 0)
df_train['PromoInterval_Feb'] = np.where(df_train['PromoInterval'].str.contains('Feb'), 1, 0)
df_train['PromoInterval_Mar'] = np.where(df_train['PromoInterval'].str.contains('Mar'), 1, 0)
df_train['PromoInterval_Apr'] = np.where(df_train['PromoInterval'].str.contains('Apr'), 1, 0)
df_train['PromoInterval_May'] = np.where(df_train['PromoInterval'].str.contains('May'), 1, 0)
df_train['PromoInterval_Jun'] = np.where(df_train['PromoInterval'].str.contains('Jun'), 1, 0)
df_train['PromoInterval_Jul'] = np.where(df_train['PromoInterval'].str.contains('Jul'), 1, 0)
df_train['PromoInterval_Aug'] = np.where(df_train['PromoInterval'].str.contains('Aug'), 1, 0)
df_train['PromoInterval_Sept'] = np.where(df_train['PromoInterval'].str.contains('Sept'), 1, 0)
df_train['PromoInterval_Oct'] = np.where(df_train['PromoInterval'].str.contains('Oct'), 1, 0)
df_train['PromoInterval_Nov'] = np.where(df_train['PromoInterval'].str.contains('Nov'), 1, 0)
df_train['PromoInterval_Dec'] = np.where(df_train['PromoInterval'].str.contains('Dec'), 1, 0)
df_train = df_train.drop(['PromoInterval'], axis=1)


In [10]:
df_train = df_train.fillna(0)

In [11]:
# Sort the DataFrame by Store and Date
df_train = df_train.sort_values(['Store', 'Date'])

# Calculate the last week same day sales
df_train['Sales_LastWeek'] = df_train.groupby(['Store'])['Sales'].transform(lambda x: x.shift(7))
df_train['Sales_LastMonth'] = df_train.groupby(['Store'])['Sales'].transform(lambda x: x.shift(30))

df_train['Customers_LastWeek'] = df_train.groupby(['Store'])['Customers'].transform(lambda x: x.shift(7))
df_train['Customers_LastMonth'] = df_train.groupby(['Store'])['Customers'].transform(lambda x: x.shift(30))

In [12]:
# Calculate the total sales for the last week
df_train['TotalSales_LastWeek'] = df_train.groupby(['Store', 'Month'])['Sales'].transform(lambda x: x.rolling(window=7).sum().shift(1))
df_train['MeanSales_LastWeek'] = df_train.groupby(['Store', 'Month'])['Sales'].transform(lambda x: x.rolling(window=7).mean().shift(1))
df_train['MedianSales_LastWeek'] = df_train.groupby(['Store', 'Month'])['Sales'].transform(lambda x: x.rolling(window=7).median().shift(1))
df_train['MinSales_LastWeek'] = df_train.groupby(['Store', 'Month'])['Sales'].transform(lambda x: x.rolling(window=7).min().shift(1))
df_train['MaxSales_LastWeek'] = df_train.groupby(['Store', 'Month'])['Sales'].transform(lambda x: x.rolling(window=7).max().shift(1))
df_train['StdSales_LastWeek'] = df_train.groupby(['Store', 'Month'])['Sales'].transform(lambda x: x.rolling(window=7).std().shift(1))

df_train['TotalCustomers_LastWeek'] = df_train.groupby(['Store', 'Month'])['Customers'].transform(lambda x: x.rolling(window=7).sum().shift(1))
df_train['MeanCustomers_LastWeek'] = df_train.groupby(['Store', 'Month'])['Customers'].transform(lambda x: x.rolling(window=7).mean().shift(1))
df_train['MedianCustomers_LastWeek'] = df_train.groupby(['Store', 'Month'])['Customers'].transform(lambda x: x.rolling(window=7).median().shift(1))
df_train['MinCustomers_LastWeek'] = df_train.groupby(['Store', 'Month'])['Customers'].transform(lambda x: x.rolling(window=7).min().shift(1))
df_train['MaxCustomers_LastWeek'] = df_train.groupby(['Store', 'Month'])['Customers'].transform(lambda x: x.rolling(window=7).max().shift(1))
df_train['StdCustomers_LastWeek'] = df_train.groupby(['Store', 'Month'])['Customers'].transform(lambda x: x.rolling(window=7).std().shift(1))



In [13]:
df_train = df_train.dropna()

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 882294 entries, 967035 to 1114
Data columns (total 53 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      882294 non-null  int64         
 1   DayOfWeek                  882294 non-null  int64         
 2   Date                       882294 non-null  datetime64[ns]
 3   Sales                      882294 non-null  int64         
 4   Customers                  882294 non-null  int64         
 5   Open                       882294 non-null  int64         
 6   Promo                      882294 non-null  int64         
 7   SchoolHoliday              882294 non-null  int64         
 8   Day                        882294 non-null  int32         
 9   Month                      882294 non-null  int32         
 10  Quarter                    882294 non-null  int32         
 11  CompetitionDistance        882294 non-null  float64   

In [16]:
df_train.to_csv('../data/processed/rossmann_sales_df.csv', index=False)